In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xarray as xr
from dateutil.relativedelta import relativedelta

from mapper_functions import plot_global_tight, plot_global_tight_pcm, plot_sa_tight_pcm, plot_tb_tight_pcm

import cartopy.crs as ccrs

In [ ]:
ds = xr.open_dataset('DAv7_M36.inst3_1d_lndfcstana_Nt.20150901.nc4')
lon = ds['lon']
lat = ds['lat']
n_tile = len(lat)

In [ ]:
start_date = datetime(2003, 1, 1)
end_date = datetime(2004, 1, 1)

start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

In [ ]:
expt_name = 'snow_LS_DAv8_M36'

data = np.load(f'../test_data/snow_da/{expt_name}_{start_date_str}_{end_date_str}_catch_progn_incr_stats_v3.npz')

ex_number_of_incr      = data['number_of_incr']
ex_std_sfexc_increment = data['ts_std_surfexc_incr']
ex_std_rzexc_increment = data['ts_std_rzexc_incr']
ex_std_total_increment = data['ts_std_total_incr']

ex_number_of_snow_mass_incr = data['num_snow_incr']
ex_std_snow_mass_increment = data['ts_std_snow_incr']
ex_mean_snow_mass_increment = data['ts_mean_snow_incr']

In [ ]:
obarray = np.empty([n_tile, 3])
obarray.fill(np.nan)
obarray[:, 1] = lon
obarray[:, 2] = lat

In [ ]:
obarray[:, 0] = ex_number_of_snow_mass_incr
obarray[:, 0][obarray[:, 0] == 0] = np.nan
maxval = np.nanmax(obarray[:, 0]) 
minval = np.nanmin(obarray[:, 0]) 
plot_global_tight_pcm(obarray,False, False,f'{expt_name} 1/2003 - 1/2004:\n Number of snow mass increments (Max: {maxval:.3g} Min: {minval:.3g})','Incrs', 1, 800)


In [ ]:
plot_tb_tight_pcm(obarray,False, False,f'{expt_name} 1/2003 - 1/2004:\n Number of snow mass increments (Max: {maxval:.3g} Min: {minval:.3g})','Incrs', 1, 800)


In [ ]:
lon_value = 84.0
lat_value = 31.0

# find the index of the tile closest to the given lat/lon
tile_index = np.argmin((lat.values - lat_value)**2 + (lon.values - lon_value)**2)
print(tile_index)

In [ ]:
# Load all the concatenated observations
data = np.load(f'../test_data/snow_da/{expt_name}_{start_date_str}_{end_date_str}_obsfcstana_extend_datetime_2003.npz', allow_pickle=True)

# List of all the variables in the file
data.files

In [ ]:
ts_ts = data['date_time'][data['obs_tilenum'] == tile_index]
ts_obs = data['obs_obs'][data['obs_tilenum'] == tile_index]
ts_fcst = data['obs_fcst'][data['obs_tilenum'] == tile_index]
ts_ana = data['obs_ana'][data['obs_tilenum'] == tile_index]

In [ ]:
print(ts_ts[1])

# Parse time information and create datetime objects
dates = []
for entry in ts_ts:
    year = entry['year'][0]
    month = entry['month'][0]
    day = entry['day'][0]
    hour = entry['hour'][0]
    minute = entry['min'][0]
    second = entry['sec'][0]
    dates.append(datetime(year, month, day, hour, minute, second))

In [ ]:
# Sort dates, ts_obs, ts_fcst, ts_ana by date
dates, ts_obs, ts_fcst, ts_ana = zip(*sorted(zip(dates, ts_obs, ts_fcst, ts_ana)))

# Ensure ts_ana and ts_fcst are numerical arrays
ts_obs = np.array(ts_obs)
ts_ana = np.array(ts_ana)
ts_fcst = np.array(ts_fcst)

In [ ]:
bar_width = 0.15
opacity = 0.9

# Plot obs_inc_s in the top subplot with two y-axes
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(20, 10), sharex=True)
ax1.bar(dates, ts_obs, bar_width, alpha=opacity, color='blue', label='SCF')
ax1.set_ylabel('SCF Obs (-)', weight='bold', fontsize=14)
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title(f'{expt_name} 1/2003 - 1/2004: Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax1.grid(axis='y')
ax1.legend(loc='upper left')

ax2.bar(dates, ts_ana - ts_fcst, bar_width, alpha=opacity, color='green', label='SCF FCST')
#ax2.bar(dates, ts_ana, bar_width, alpha=opacity, color='red', label='SCF ANA')
ax2.set_ylabel('SCF ANA - FCST (-)', weight='bold', fontsize=14)
ax2.tick_params(axis='y', labelcolor='green')
ax2.grid(axis='y')
ax2.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

ax.bar(dates, ts_obs, bar_width, alpha=opacity, label='SCF Obs', color='blue')
ax.plot(dates, ts_fcst, label='SCF FCST', color='red')
ax.plot(dates, ts_ana, label='SCF ANA', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('SCF Values (-)', weight='bold', fontsize=14)
ax.set_title(f'{expt_name} 1/2003 - 1/2004: Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)

plt.show()

In [ ]:
# Define the new experiment name
expt_name_2 = 'snow_LS_OLv8_M36'

# Load the data for the new experiment
data_2 = np.load(f'../test_data/snow_da/{expt_name_2}_{start_date_str}_{end_date_str}_obsfcstana_extend_datetime_2003.npz', allow_pickle=True)

# Extract the relevant data for the new experiment
ts_ts_2 = data_2['date_time'][data_2['obs_tilenum'] == tile_index]
ts_obs_2 = data_2['obs_obs'][data_2['obs_tilenum'] == tile_index]
ts_fcst_2 = data_2['obs_fcst'][data_2['obs_tilenum'] == tile_index]
ts_ana_2 = data_2['obs_ana'][data_2['obs_tilenum'] == tile_index]

# Parse time information and create datetime objects for the new experiment
dates_2 = []
for entry in ts_ts_2:
    year = entry['year'][0]
    month = entry['month'][0]
    day = entry['day'][0]
    hour = entry['hour'][0]
    minute = entry['min'][0]
    second = entry['sec'][0]
    dates_2.append(datetime(year, month, day, hour, minute, second))

# Sort dates, ts_obs_2, ts_fcst_2, ts_ana_2 by date
dates_2, ts_obs_2, ts_fcst_2, ts_ana_2 = zip(*sorted(zip(dates_2, ts_obs_2, ts_fcst_2, ts_ana_2)))

# Ensure ts_ana_2 and ts_fcst_2 are numerical arrays
ts_obs_2 = np.array(ts_obs_2)
ts_ana_2 = np.array(ts_ana_2)
ts_fcst_2 = np.array(ts_fcst_2)

# Plot the data for the new experiment
fig, ax = plt.subplots(figsize=(20, 10))

ax.bar(dates_2, ts_obs_2, bar_width, alpha=opacity, label='SCF Obs', color='blue')
ax.plot(dates_2, ts_fcst_2, label='SCF FCST', color='red')
ax.plot(dates_2, ts_ana_2, label='SCF ANA', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('SCF Values (-)', weight='bold', fontsize=14)
ax.set_title(f'{expt_name_2} 1/2003 - 1/2004: Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

# Plot the observations as bars
ax.bar(dates, ts_obs, bar_width, alpha=opacity, label='SCF Obs', color='blue')

# Plot the first forecast
ax.plot(dates, ts_fcst, label=f'{expt_name} SCF FCST', color='orange')

# Plot the second forecast
ax.plot(dates_2, ts_fcst_2, label=f'{expt_name_2} SCF FCST', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('SCF Values (-)', weight='bold', fontsize=14)
ax.set_title(f'Comparison of SCF Forecasts and Observations: Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)

plt.show()

In [ ]:
# Construct the name using expt_name and start/end date
constructed_name = f'{expt_name}_{start_date_str}_{end_date_str}_catch_progn_incr_concat_snow_only'
print(f'Constructed name: {constructed_name}')

# Load the specified .npz file using the constructed name
data_snow_only = np.load(f'../test_data/snow_da/{constructed_name}.npz')

# Print the contents of the .npz file
print(data_snow_only.files)

# Parse the timestamps to datetime objects
dates_concat = [datetime.strptime(ts.decode('utf-8'), '%Y%m%d_%H%Mz') for ts in data_snow_only['time_stamp_concat']]

# Check what dates_concat looks like
print(dates_concat[:10])

# Check the shape of snow_incr_concat in the .npz file
print(data_snow_only['snow_incr_concat'].shape)

# Exctract the snow increments for the tile of interest
snow_incr_tile = data_snow_only['snow_incr_concat'][:, tile_index]

# Plot the snow increments for the tile of interest
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(dates_concat, snow_incr_tile, label='Snow Increments')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Snow Increments (mm)', weight='bold', fontsize=14)
ax.set_title(f'Snow Increments for Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)


In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 8))

color = 'tab:blue'
ax1.set_xlabel('Date', weight='bold', fontsize=14)
ax1.set_ylabel('Snow Increments (mm)', color=color, weight='bold', fontsize=14)
ax1.plot(dates_concat, snow_incr_tile, label='Snow Increments', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')
ax1.grid(True)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax2.set_ylabel('SCF ANA - FCST', color=color, weight='bold', fontsize=14)  # we already handled the x-label with ax1
ax2.plot(dates, ts_ana - ts_fcst, label='SCF ANA', color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')
ax1.set_ylim(-2, 4)
ax2.set_ylim(-0.2, 0.4)


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title(f'Snow Increments and SCF ANA for Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
plt.show()

In [ ]:
# Filter the data for the first three months
end_date_three_months = start_date + relativedelta(months=2)
mask = (np.array(dates_concat) >= start_date) & (np.array(dates_concat) < end_date_three_months)

dates_concat_3months = np.array(dates_concat)[mask]
snow_incr_tile_3months = snow_incr_tile[mask]

mask_ana_fcst = (np.array(dates) >= start_date) & (np.array(dates) < end_date_three_months)
dates_3months = np.array(dates)[mask_ana_fcst]
ts_ana_3months = ts_ana[mask_ana_fcst]
ts_fcst_3months = ts_fcst[mask_ana_fcst]

fig, ax1 = plt.subplots(figsize=(20, 8))

color = 'tab:blue'
ax1.set_xlabel('Date', weight='bold', fontsize=14)
ax1.set_ylabel('Snow Increments (mm)', color=color, weight='bold', fontsize=14)
ax1.bar(dates_concat_3months, snow_incr_tile_3months, alpha=opacity, label='Snow Increments', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')
ax1.grid(True)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax2.set_ylabel('SCF ANA - FCST', color=color, weight='bold', fontsize=14)  # we already handled the x-label with ax1
ax2.plot(dates_3months, ts_ana_3months - ts_fcst_3months, label='SCF ANA', color=color, marker='o', linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')
ax1.set_ylim(-2, 4)
ax2.set_ylim(-0.2, 0.4)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title(f'Snow Increments and SCF ANA for Tile {tile_index} ({lon_value}, {lat_value}) - First Two Months', weight='bold', fontsize=16)
plt.show()

In [ ]:
# Load the specified .npz file
OL_tavg24 = np.load('../test_data/snow_da/snow_LS_OLv8_M36_20030101_20040101_tavg24_1d_lnd_Nt_concat.npz')
DA_tavg24 = np.load('../test_data/snow_da/snow_LS_DAv8_M36_20030101_20040101_tavg24_1d_lnd_Nt_concat.npz')

# Print the contents of the .npz file
for file in OL_tavg24.files:
    print(file)

# Time series of snow_mass_concat at tile_index for OL and DA, using dates from above
snow_mass_OL = OL_tavg24['snow_mass_concat'][:, tile_index]
snow_mass_DA = DA_tavg24['snow_mass_concat'][:, tile_index]

# Make a daily timestamp array for the concatenated data for 2023
daily_dates = [datetime(2003, 1, 1) + timedelta(days=i) for i in range(len(snow_mass_OL))]
print(dates_concat[:10])

# Plot the time series of snow_mass_concat at tile_index for OL and DA
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(daily_dates, snow_mass_OL, label='OL Snow Mass')
ax.plot(daily_dates, snow_mass_DA, label='DA Snow Mass')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Snow Mass (kg m^-^2)', weight='bold', fontsize=14)
ax.set_title(f'Snow Mass for Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
plt.show()

# Extract precipitation_total_surface_flux_concat for DA
precip_DA = DA_tavg24['precipitation_total_surface_flux_concat'][:, tile_index]

# Convert to daily from seconds
precip_DA_daily = precip_DA * 86400

# Plot the time series of precipitation_total_surface_flux_concat for DA
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(daily_dates, precip_DA_daily, label='DA Precipitation')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Precipitation (mm)', weight='bold', fontsize=14)
ax.set_title(f'Precipitation for Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
plt.show()

In [ ]:
# Calculate cumulative snow increments and precipitation
cumulative_snow_incr = np.cumsum(snow_incr_tile)
cumulative_precip_DA = np.cumsum(precip_DA_daily)

# Plot cumulative snow increments and precipitation on a single Y axis
fig, ax = plt.subplots(figsize=(20, 10))

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.plot(dates_concat, cumulative_snow_incr, label='Cumulative Snow Increments', color='tab:blue')
ax.plot(daily_dates, cumulative_precip_DA, label='Cumulative Precipitation', color='tab:green')
ax.tick_params(axis='y')
ax.legend(loc='upper left')
ax.grid(True)

plt.title(f'Cumulative Snow Increments and Precipitation for Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
plt.show()


In [ ]:
# Extract sm_profile_concat for OL and DA
sm_profile_OL = OL_tavg24['sm_profile_concat'][:, tile_index]
sm_profile_DA = DA_tavg24['sm_profile_concat'][:, tile_index]

# Plot the time series of sm_profile_concat at tile_index for OL and DA
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(daily_dates, sm_profile_OL, label='OL Soil Moisture Profile')
ax.plot(daily_dates, sm_profile_DA, label='DA Soil Moisture Profile')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Soil Moisture Profile (m^3 m^-^3)', weight='bold', fontsize=14)
ax.set_title(f'Soil Moisture Profile for Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
plt.show()

In [ ]:
# Assuming OL_tavg24 is a dictionary or similar structure containing the variables

# Define the tile index and dates (example)
# tile_index = 0  # Replace with the actual tile index you are interested in
start_date = datetime(2003, 1, 1)
end_date = datetime(2004, 1, 1)
daily_dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days)]

# Extract the data for the tile of interest using OL_tavg24
sm_surface = OL_tavg24['sm_surface_concat'][:, tile_index]
sm_rootzone = OL_tavg24['sm_rootzone_concat'][:, tile_index]
sm_profile = OL_tavg24['sm_profile_concat'][:, tile_index]
precipitation_total_surface_flux = OL_tavg24['precipitation_total_surface_flux_concat'][:, tile_index] * 86400  # Convert to daily from seconds
vegetation_greenness_fraction = OL_tavg24['vegetation_greenness_fraction_concat'][:, tile_index]
leaf_area_index = OL_tavg24['leaf_area_index_concat'][:, tile_index]
snow_mass = OL_tavg24['snow_mass_concat'][:, tile_index]
surface_temperature_of_land_incl_snow = OL_tavg24['surface_temperature_of_land_incl_snow_concat'][:, tile_index]
soil_temperature_layer_1 = OL_tavg24['soil_temperature_layer_1_concat'][:, tile_index]
snowfall_land = OL_tavg24['snowfall_land_concat'][:, tile_index] * 86400  # Convert to daily from seconds
snow_depth_within_snow_covered_area_fraction_on_land = OL_tavg24['snow_depth_within_snow_covered_area_fraction_on_land_concat'][:, tile_index]
snowpack_evaporation_latent_heat_flux_on_land = OL_tavg24['snowpack_evaporation_latent_heat_flux_on_land_concat'][:, tile_index] * 0.03456 # Convert to mm per day
overland_runoff_including_throughflow = OL_tavg24['overland_runoff_including_throughflow_concat'][:, tile_index] * 86400  # Convert to daily from seconds
baseflow_flux_land = OL_tavg24['baseflow_flux_land_concat'][:, tile_index] * 86400  # Convert to daily from seconds
snowmelt_flux_land = OL_tavg24['snowmelt_flux_land_concat'][:, tile_index] * 86400  # Convert to daily from seconds
total_evaporation_land = OL_tavg24['total_evaporation_land_concat'][:, tile_index] * 86400  # Convert to daily from seconds
net_shortwave_flux_land = OL_tavg24['net_shortwave_flux_land_concat'][:, tile_index]
total_water_storage_land = OL_tavg24['total_water_storage_land_concat'][:, tile_index]


In [ ]:

# Plot the water balance components
fig, ax1 = plt.subplots(figsize=(20, 10))

color = 'tab:blue'
ax1.set_xlabel('Date', weight='bold', fontsize=14)
ax1.set_ylabel('Precipitation and Evaporation (mm/day)', color=color, weight='bold', fontsize=14)
ax1.plot(daily_dates, precipitation_total_surface_flux, label='Precipitation', color='blue')
ax1.plot(daily_dates, total_evaporation_land, label='Evaporation', color='red')
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')
ax1.grid(True)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax2.set_ylabel('Runoff, Baseflow, and Snowmelt (mm/day)', color=color, weight='bold', fontsize=14)
ax2.plot(daily_dates, overland_runoff_including_throughflow, label='Runoff', color='green')
ax2.plot(daily_dates, baseflow_flux_land, label='Baseflow', color='purple')
ax2.plot(daily_dates, snowmelt_flux_land, label='Snowmelt', color='cyan')
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title(f'Water Balance Components for Tile {tile_index}', weight='bold', fontsize=16)
plt.show()

# Plot soil moisture and snow mass
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, sm_surface, label='Surface Soil Moisture', color='brown')
ax.plot(daily_dates, sm_rootzone, label='Rootzone Soil Moisture', color='orange')
ax.plot(daily_dates, sm_profile, label='Profile Soil Moisture', color='yellow')
ax.plot(daily_dates, snow_mass, label='Snow Mass', color='cyan')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Soil Moisture and Snow Mass', weight='bold', fontsize=14)
ax.set_title(f'Soil Moisture and Snow Mass for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)

plt.show()

# Plot total water storage
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, total_water_storage_land, label='Total Water Storage', color='blue')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Total Water Storage (mm)', weight='bold', fontsize=14)
ax.set_title(f'Total Water Storage for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)

plt.show()

In [ ]:
# Plot total precipitation and snowfall
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, precip_DA_daily, label='Total Precipitation', color='blue')
ax.plot(daily_dates, snowfall_land, label='Snowfall', color='cyan')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Precipitation and Snowfall (mm)', weight='bold', fontsize=14)
ax.set_title(f'Total Precipitation and Snowfall for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

# Plot snow mass and snow melt
fig, ax = plt.subplots(figsize=(20, 10))

ax2 = ax.twinx()
ax2.plot(daily_dates, snow_mass, label='Snow Mass', color='blue')
ax2.set_ylabel('Snow Mass (mm)', weight='bold', fontsize=14)
ax2.tick_params(axis='y', labelcolor='blue')
ax2.legend(loc='upper right', fontsize=14)
ax.plot(daily_dates, snowmelt_flux_land, label='Snow Melt', color='cyan')
ax.plot(daily_dates, snowpack_evaporation_latent_heat_flux_on_land, label='Snowpack Evaporation', color='green')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Snow Melt (mm)', weight='bold', fontsize=14)
ax.set_title(f'Snow Mass and Snow Melt for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

# Calculate cumulative snowfall and cumulative snowmelt + snowpack evaporation
cumulative_snowfall = np.cumsum(snowfall_land)
cumulative_snowmelt_evaporation = np.cumsum(snowmelt_flux_land + snowpack_evaporation_latent_heat_flux_on_land)


# Plot cumulative snowfall and cumulative snowmelt + snowpack evaporation
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, cumulative_snowfall, label='Cumulative Snowfall', color='blue')
ax.plot(daily_dates, cumulative_snowmelt_evaporation, label='Cumulative Snowmelt + Evaporation', color='cyan')
ax.plot(daily_dates, snow_mass, label='Snow Mass', color='green')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Snowfall, Snowmelt + Evaporation, and Daily Change in Snow Mass for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

# Plot the snow depth within snow covered area fraction on land
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, snow_depth_within_snow_covered_area_fraction_on_land, label='Snow Depth Fraction', color='blue')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Snow Depth within SCF', weight='bold', fontsize=14)
ax.set_title(f'Snow Depth within SCF for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

# Plot snowfall as a bar graph and snowmelt + snow evaporation as lines
fig, ax = plt.subplots(figsize=(20, 10))

ax.bar(daily_dates, snowfall_land, label='Snowfall', color='blue', alpha=0.6)
ax.plot(daily_dates, snowmelt_flux_land, label='Snowmelt', color='cyan', linewidth=2)
ax.plot(daily_dates, snowpack_evaporation_latent_heat_flux_on_land, label='Snow Evaporation', color='green', linewidth=2)
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Snowfall, Snowmelt, and Snow Evaporation for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

In [ ]:
# Calculate cumulative values for the open loop experiment
cumulative_precip_OL = np.cumsum(precipitation_total_surface_flux)
cumulative_evap_OL = np.cumsum(total_evaporation_land)
cumulative_runoff_OL = np.cumsum(overland_runoff_including_throughflow)
cumulative_baseflow_OL = np.cumsum(baseflow_flux_land)

# Plot cumulative values
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, cumulative_precip_OL, label='Cumulative Precipitation', color='blue')
ax.plot(daily_dates, cumulative_evap_OL, label='Cumulative Evaporation', color='red')
ax.plot(daily_dates, cumulative_runoff_OL, label='Cumulative Runoff', color='green')
ax.plot(daily_dates, cumulative_baseflow_OL, label='Cumulative Baseflow', color='purple')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Precipitation, Evaporation, Runoff, and Baseflow for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

In [ ]:
# Calculate cumulative values for the DA experiment
cumulative_precip_DA = np.cumsum(precip_DA_daily)
cumulative_evap_DA = np.cumsum(DA_tavg24['total_evaporation_land_concat'][:, tile_index] * 86400)
cumulative_runoff_DA = np.cumsum(DA_tavg24['overland_runoff_including_throughflow_concat'][:, tile_index] * 86400)
cumulative_baseflow_DA = np.cumsum(DA_tavg24['baseflow_flux_land_concat'][:, tile_index] * 86400)

# Plot cumulative values
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, cumulative_precip_DA, label='Cumulative Precipitation (DA)', color='blue')
ax.plot(daily_dates, cumulative_evap_DA, label='Cumulative Evaporation (DA)', color='red')
ax.plot(daily_dates, cumulative_runoff_DA, label='Cumulative Runoff (DA)', color='green')
ax.plot(daily_dates, cumulative_baseflow_DA, label='Cumulative Baseflow (DA)', color='purple')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Precipitation, Evaporation, Runoff, and Baseflow for Tile {tile_index} (DA)', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

# Plot cumulative precipitation
ax.plot(daily_dates, cumulative_precip_OL, label='Cumulative Precipitation (OL)', color='blue', linestyle='--')
ax.plot(daily_dates, cumulative_precip_DA, label='Cumulative Precipitation (DA)', color='blue')

# Plot cumulative evaporation
ax.plot(daily_dates, cumulative_evap_OL, label='Cumulative Evaporation (OL)', color='red', linestyle='--')
ax.plot(daily_dates, cumulative_evap_DA, label='Cumulative Evaporation (DA)', color='red')

# Plot cumulative runoff
ax.plot(daily_dates, cumulative_runoff_OL, label='Cumulative Runoff (OL)', color='green', linestyle='--')
ax.plot(daily_dates, cumulative_runoff_DA, label='Cumulative Runoff (DA)', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Precipitation, Evaporation, and Runoff for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

In [ ]:
# Calculate cumulative snowfall, snowmelt, and snow evaporation for OL
cumulative_snowfall_OL = np.cumsum(snowfall_land)
cumulative_snowmelt_OL = np.cumsum(snowmelt_flux_land)
cumulative_snow_evap_OL = np.cumsum(snowpack_evaporation_latent_heat_flux_on_land)

# Calculate cumulative snowfall, snowmelt, and snow evaporation for DA
cumulative_snowfall_DA = np.cumsum(DA_tavg24['snowfall_land_concat'][:, tile_index] * 86400)
cumulative_snowmelt_DA = np.cumsum(DA_tavg24['snowmelt_flux_land_concat'][:, tile_index] * 86400)
cumulative_snow_evap_DA = np.cumsum(DA_tavg24['snowpack_evaporation_latent_heat_flux_on_land_concat'][:, tile_index] * 0.03456)

# Plot cumulative snowfall, snowmelt, and snow evaporation for OL and DA
fig, ax = plt.subplots(figsize=(20, 10))

# Plot cumulative snowfall
ax.plot(daily_dates, cumulative_snowfall_OL, label='Cumulative Snowfall (OL)', color='blue', linestyle='--')
ax.plot(dates_concat, cumulative_snow_incr, label='Cumulative Snow Increments', color='red')

# Plot cumulative snowmelt
ax.plot(daily_dates, cumulative_snowmelt_OL, label='Cumulative Snowmelt (OL)', color='cyan', linestyle='--')
ax.plot(daily_dates, cumulative_snowmelt_DA, label='Cumulative Snowmelt (DA)', color='cyan')

# Plot cumulative snow evaporation
ax.plot(daily_dates, cumulative_snow_evap_OL, label='Cumulative Snow Evaporation (OL)', color='green', linestyle='--')
ax.plot(daily_dates, cumulative_snow_evap_DA, label='Cumulative Snow Evaporation (DA)', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Snowfall, Snowmelt, and Snow Evaporation for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

In [ ]:
# Sum the increments for each day
daily_increments = []
unique_days = np.unique([date.date() for date in dates_concat])
for unique_day in unique_days:
    mask = [date.date() == unique_day for date in dates_concat]
    daily_increments.append(np.sum(snow_incr_tile[mask]))
    
print("First unique day:", unique_days[0])
print("Last unique day:", unique_days[-1])

# Pad the start of daily_increments with a zero
daily_increments.insert(0, 0)

# Convert to numpy array
daily_increments = np.array(daily_increments)

# Ensure the lengths match
if len(daily_increments) != len(snowfall_land):
    print("Length mismatch between daily increments and snowfall data.")
    print("Daily increments length:", len(daily_increments))
    print("Snowfall data length:", len(snowfall_land))
else:
    # Add the daily increments to the snowfall
    snowfall_plus_increments = snowfall_land + daily_increments

    # Plot the result
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.plot(daily_dates, snowfall_plus_increments, label='Snowfall + Increments', color='blue')
    ax.set_xlabel('Date', weight='bold', fontsize=14)
    ax.set_ylabel('Snowfall + Increments (mm)', weight='bold', fontsize=14)
    ax.set_title(f'Snowfall + Increments for Tile {tile_index}', weight='bold', fontsize=16)
    ax.legend(loc='upper left', fontsize=14)
    ax.grid(True)
    plt.show()

In [ ]:
# Extract fractional_area_of_snow_on_land_concat for OL and DA
fractional_area_snow_OL = OL_tavg24['fractional_area_of_snow_on_land_concat'][:, tile_index]
fractional_area_snow_DA = DA_tavg24['fractional_area_of_snow_on_land_concat'][:, tile_index]

# Plot the fractional area of snow on land for OL and DA
fig, ax1 = plt.subplots(figsize=(20, 10))

ax1.plot(daily_dates, fractional_area_snow_OL, label='SCF (OL)', color='blue', linestyle='--')
ax1.plot(daily_dates, fractional_area_snow_DA, label='SCF (DA)', color='blue')

ax1.set_xlabel('Date', weight='bold', fontsize=14)
ax1.set_ylabel('SCF', weight='bold', fontsize=14)
ax1.set_title(f'SCF for Tile {tile_index}', weight='bold', fontsize=16)
ax1.legend(loc='upper left', fontsize=14)
ax1.grid(True)
ax1.set_ylim(bottom=0)  # Set zero as the minimum for the first y-axis

# Create a second y-axis for the snowfall
ax2 = ax1.twinx()
ax2.bar(daily_dates, snowfall_land, label='Snowfall', color='orange', alpha=0.6)
ax2.set_ylabel('Snowfall (mm)', weight='bold', fontsize=14)
ax2.tick_params(axis='y', labelcolor='orange')
ax2.legend(loc='upper right', fontsize=14)
ax2.set_ylim(bottom=0)  # Set zero as the minimum for the second y-axis

plt.show()

In [ ]:
cumulative_precip_plus_incr_DA = np.cumsum(precip_DA_daily + daily_increments)

fig, ax = plt.subplots(figsize=(20, 10))

# Plot cumulative precipitation
ax.plot(daily_dates, cumulative_precip_OL, label='Cumulative Precipitation (OL)', color='blue', linestyle='--')
ax.plot(daily_dates, cumulative_precip_plus_incr_DA, label='Cumulative Precip + Incr (DA)', color='blue')

# Plot cumulative evaporation
ax.plot(daily_dates, cumulative_evap_OL, label='Cumulative Evaporation (OL)', color='red', linestyle='--')
ax.plot(daily_dates, cumulative_evap_DA, label='Cumulative Evaporation (DA)', color='red')

# Plot cumulative runoff
ax.plot(daily_dates, cumulative_runoff_OL, label='Cumulative Runoff (OL)', color='green', linestyle='--')
ax.plot(daily_dates, cumulative_runoff_DA, label='Cumulative Runoff (DA)', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Precipitation, Evaporation, and Runoff for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

In [ ]:
# Calculate cumulative snowfall, snowmelt, and snow evaporation for OL
cumulative_snowfall_OL = np.cumsum(snowfall_land)
cumulative_snowfall_plus_incr_DA = np.cumsum(snowfall_plus_increments)
cumulative_snowmelt_OL = np.cumsum(snowmelt_flux_land)
cumulative_snow_evap_OL = np.cumsum(snowpack_evaporation_latent_heat_flux_on_land)

# Calculate cumulative snowfall, snowmelt, and snow evaporation for DA
cumulative_snowfall_DA = np.cumsum(DA_tavg24['snowfall_land_concat'][:, tile_index] * 86400)
cumulative_snowmelt_DA = np.cumsum(DA_tavg24['snowmelt_flux_land_concat'][:, tile_index] * 86400)
cumulative_snow_evap_DA = np.cumsum(DA_tavg24['snowpack_evaporation_latent_heat_flux_on_land_concat'][:, tile_index] * 0.03456)

# Plot cumulative snowfall, snowmelt, and snow evaporation for OL and DA
fig, ax = plt.subplots(figsize=(20, 10))

# Plot cumulative snowfall
ax.plot(daily_dates, cumulative_snowfall_OL, label='Cumulative Snowfall (OL)', color='blue', linestyle='--')
ax.plot(daily_dates, cumulative_snowfall_plus_incr_DA, label='Cumulative Snow Increments', color='blue')

# Plot cumulative snow evaporation
ax.plot(daily_dates, cumulative_snow_evap_OL, label='Cumulative Snow Evaporation (OL)', color='red', linestyle='--')
ax.plot(daily_dates, cumulative_snow_evap_DA, label='Cumulative Snow Evaporation (DA)', color='red')

# Plot cumulative snowmelt
ax.plot(daily_dates, cumulative_snowmelt_OL, label='Cumulative Snowmelt (OL)', color='green', linestyle='--')
ax.plot(daily_dates, cumulative_snowmelt_DA, label='Cumulative Snowmelt (DA)', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Snowfall, Snowmelt, and Snow Evaporation for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

In [ ]:
obarray[:, 0] = ex_number_of_snow_mass_incr
obarray[:, 0][obarray[:, 0] == 0] = np.nan
maxval = np.nanmax(obarray[:, 0]) 
minval = np.nanmin(obarray[:, 0]) 
plot_global_tight_pcm(obarray,False, False,f'{expt_name} 1/2003 - 1/2004:\n Number of snow mass increments (Max: {maxval:.3g} Min: {minval:.3g})','Incrs', 1, 800)

##
plot_tb_tight_pcm(obarray,False, False,f'{expt_name} 1/2003 - 1/2004:\n Number of snow mass increments (Max: {maxval:.3g} Min: {minval:.3g})','Incrs', 1, 800)

##
bar_width = 0.15
opacity = 0.9

# Plot obs_inc_s in the top subplot with two y-axes
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(20, 10), sharex=True)
ax1.bar(dates, ts_obs, bar_width, alpha=opacity, color='blue', label='SCF')
ax1.set_ylabel('SCF Obs (-)', weight='bold', fontsize=14)
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title(f'{expt_name} 1/2003 - 1/2004: Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax1.grid(axis='y')
ax1.legend(loc='upper left')

ax2.bar(dates, ts_ana - ts_fcst, bar_width, alpha=opacity, color='green', label='SCF FCST')
#ax2.bar(dates, ts_ana, bar_width, alpha=opacity, color='red', label='SCF ANA')
ax2.set_ylabel('SCF ANA - FCST (-)', weight='bold', fontsize=14)
ax2.tick_params(axis='y', labelcolor='green')
ax2.grid(axis='y')
ax2.legend(loc='upper left')

##
# Plot the snow increments for the tile of interest
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(dates_concat, snow_incr_tile, label='Snow Increments')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Snow Increments (mm)', weight='bold', fontsize=14)
ax.set_title(f'Snow Increments for Tile {tile_index} ({lon_value}, {lat_value})', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)

# Plot the fractional area of snow on land for OL and DA
fig, ax1 = plt.subplots(figsize=(20, 10))

ax1.plot(daily_dates, fractional_area_snow_OL, label='SCF (OL)', color='blue', linestyle='--')
ax1.plot(daily_dates, fractional_area_snow_DA, label='SCF (DA)', color='blue')

ax1.set_xlabel('Date', weight='bold', fontsize=14)
ax1.set_ylabel('SCF', weight='bold', fontsize=14)
ax1.set_title(f'SCF for Tile {tile_index}', weight='bold', fontsize=16)
ax1.legend(loc='upper left', fontsize=14)
ax1.grid(True)
ax1.set_ylim(bottom=0)  # Set zero as the minimum for the first y-axis

# Create a second y-axis for the snowfall
ax2 = ax1.twinx()
ax2.bar(daily_dates, snowfall_land, label='Snowfall', color='orange', alpha=0.6)
ax2.set_ylabel('Snowfall (mm)', weight='bold', fontsize=14)
ax2.tick_params(axis='y', labelcolor='orange')
ax2.legend(loc='upper right', fontsize=14)
ax2.set_ylim(bottom=0)  # Set zero as the minimum for the second y-axis

plt.show()

##
fig, ax = plt.subplots(figsize=(20, 10))

# Plot cumulative precipitation
ax.plot(daily_dates, cumulative_precip_OL, label='Cumulative Precipitation (OL)', color='blue', linestyle='--')
ax.plot(daily_dates, cumulative_precip_plus_incr_DA, label='Cumulative Precip + Incr (DA)', color='blue')

# Plot cumulative evaporation
ax.plot(daily_dates, cumulative_evap_OL, label='Cumulative Evaporation (OL)', color='red', linestyle='--')
ax.plot(daily_dates, cumulative_evap_DA, label='Cumulative Evaporation (DA)', color='red')

# Plot cumulative runoff
ax.plot(daily_dates, cumulative_runoff_OL, label='Cumulative Runoff (OL)', color='green', linestyle='--')
ax.plot(daily_dates, cumulative_runoff_DA, label='Cumulative Runoff (DA)', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Precipitation, Evaporation, and Runoff for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

# Plot cumulative snowfall, snowmelt, and snow evaporation for OL and DA
fig, ax = plt.subplots(figsize=(20, 10))

# Plot cumulative snowfall
ax.plot(daily_dates, cumulative_snowfall_OL, label='Cumulative Snowfall (OL)', color='blue', linestyle='--')
ax.plot(daily_dates, cumulative_snowfall_plus_incr_DA, label='Cumulative Snowfall + Incr', color='blue')

# Plot cumulative snow evaporation
ax.plot(daily_dates, cumulative_snow_evap_OL, label='Cumulative Snow Evaporation (OL)', color='red', linestyle='--')
ax.plot(daily_dates, cumulative_snow_evap_DA, label='Cumulative Snow Evaporation (DA)', color='red')

# Plot cumulative snowmelt
ax.plot(daily_dates, cumulative_snowmelt_OL, label='Cumulative Snowmelt (OL)', color='green', linestyle='--')
ax.plot(daily_dates, cumulative_snowmelt_DA, label='Cumulative Snowmelt (DA)', color='green')

ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Cumulative Values (mm)', weight='bold', fontsize=14)
ax.set_title(f'Cumulative Snowfall, Snowmelt, and Snow Evaporation for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left', fontsize=14)
ax.grid(True)

plt.show()

# Plot total water storage for OL and DA
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(daily_dates, OL_tavg24['total_water_storage_land_concat'][:, tile_index], label='Total Water Storage (OL)')
ax.plot(daily_dates, DA_tavg24['total_water_storage_land_concat'][:, tile_index], label='Total Water Storage (DA)')
ax.set_xlabel('Date', weight='bold', fontsize=14)
ax.set_ylabel('Total Water Storage (mm)', weight='bold', fontsize=14)
ax.set_title(f'Total Water Storage for Tile {tile_index}', weight='bold', fontsize=16)
ax.legend(loc='upper left')
ax.grid(True)

plt.show()